<a href="https://colab.research.google.com/github/HrishiDhondge/NVIDIA_workshop/blob/main/Nemo_agent_toolkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook with Pre-requisites for NVIDIA workshop

In [1]:
# Download and install git-lfs
!wget https://github.com/git-lfs/git-lfs/releases/download/v3.7.0/git-lfs-linux-amd64-v3.7.0.tar.gz

# Extract the files
!tar -xvf git-lfs-linux-amd64-v3.7.0.tar.gz

# Install
!git-lfs-3.7.0/./install.sh

--2025-07-31 02:31:03--  https://github.com/git-lfs/git-lfs/releases/download/v3.7.0/git-lfs-linux-amd64-v3.7.0.tar.gz
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/13021798/7c23ad92-6754-4126-bc83-f9134c4184d4?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-07-31T03%3A16%3A19Z&rscd=attachment%3B+filename%3Dgit-lfs-linux-amd64-v3.7.0.tar.gz&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-07-31T02%3A15%3A23Z&ske=2025-07-31T03%3A16%3A19Z&sks=b&skv=2018-11-09&sig=TPcgBhDoUY%2BCbTJMpbeGrFK95ibUG2iZ3hFy0m6iaMM%3D&jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1MzkyOTM2MywibmJmIjoxNzUzOTI5MDYzLCJwYXRoIjoic

In [2]:
# Install uv with one of these two ways
#curl -LsSf https://astral.sh/uv/install.sh | sh
!pip install uv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 76.7 MB/s eta 0:00:00


In [5]:
# Clone repo
!git clone https://github.com/NVIDIA/NeMo-Agent-Toolkit.git aiqtoolkit
%cd aiqtoolkit

Cloning into 'aiqtoolkit'...
remote: Enumerating objects: 8528, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 8528 (delta 190), reused 104 (delta 96), pack-reused 8207 (from 3)
Receiving objects: 100% (8528/8528), 37.17 MiB | 30.21 MiB/s, done.
Resolving deltas: 100% (4301/4301), done.
Filtering content: 100% (64/64), 20.78 MiB | 11.89 MiB/s, done.
/content/aiqtoolkit


In [6]:
#Initialize, fetch, and update submodules
!git submodule update --init --recursive

Submodule 'external/aiqtoolkit-opensource-ui' (https://github.com/NVIDIA/NeMo-Agent-Toolkit-UI.git) registered for path 'external/aiqtoolkit-opensource-ui'
Cloning into '/content/aiqtoolkit/external/aiqtoolkit-opensource-ui'...
Submodule path 'external/aiqtoolkit-opensource-ui': checked out 'eac58dbe62200729e7bde0ca15cc27efca7b082f'


In [7]:
# Fetch the data sets
!git lfs install
!git lfs fetch
!git lfs pull

Updated Git hooks.
Git LFS initialized.
Fetching reference refs/heads/develop


In [8]:
# !rm -rf .venv/
# Creating the environment
!uv venv --seed .venv
!source .venv/bin/activate


Using CPython 3.12.11
Creating virtual environment with seed packages at: .venv
 + pip==25.2
Activate with: source .venv/bin/activate


In [9]:
# UV syncronization
!uv venv --seed .venv --python 3.11 -c
!uv sync --all-groups --all-extras

Streaming output truncated to the last 5000 lines.
⠋ Preparing packages... (347/517)
bcrypt                                      ------------------------------ 277.50 KiB/277.50 KiB
llama-cloud                                 ------------------------------ 30.91 KiB/277.95 KiB
psutil                                      ------------------------------ 78.58 KiB/283.65 KiB
anthropic                                   ------------------------------ 62.91 KiB/286.19 KiB
googleapis-common-protos                    ------------------------------ 48.00 KiB/287.63 KiB
seaborn                                     ------------------------------ 188.00 KiB/288.00 KiB
strawberry-graphql                          ------------------------------ 159.51 KiB/298.86 KiB
joblib                                      ------------------------------ 123.82 KiB/300.53 KiB
pypdf                                       ------------------------------ 238.90 KiB/302.46 KiB
dnspython                                   --

In [ ]:
!uv pip install -e '.[langchain]'
!uv pip install -e '.[llama-index]'

!uv pip install -e '.[profiling]'

# export aiq
import os
os.environ['aiq'] = './.venv/bin/aiq'
!$aiq --version


aiq, version 1.2.0rc3.dev19+gdc80c2b


In [ ]:
!export NVIDIA_API_KEY=<your_api_key>

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `export NVIDIA_API_KEY=<your_api_key>'


In [ ]:
!$aiq run --config_file workflow.yaml --input "List five subspecies of Aardvarks"

2025-07-31 07:53:45,320 - aiq.runtime.loader - WARNING - Loading module 'aiq_profiler_agent.register' from entry point 'aiq_profiler_agent' took a long time (346.319914 ms). Ensure all imports are inside your registered functions.
2025-07-31 07:53:45,424 - aiq.data_models.discovery_metadata - WARNING - Package metadata not found for simple_auth
2025-07-31 07:53:45,879 - aiq.runtime.loader - WARNING - Loading module 'aiq.agent.register' from entry point 'aiq_agents' took a long time (422.675371 ms). Ensure all imports are inside your registered functions.
2025-07-31 07:53:46,232 - aiq.runtime.loader - WARNING - Loading module 'aiq_multi_frameworks.register' from entry point 'aiq_multi_frameworks' took a long time (171.598673 ms). Ensure all imports are inside your registered functions.
2025-07-31 07:53:46,376 - aiq.cli.commands.start - INFO - Starting AIQ Toolkit from config file: 'workflow.yaml'
2025-07-31 07:53:46,378 - aiq.cli.commands.start - WARNING - The front end type in the conf